In [ ]:
# notes : the model should be robust for both male and female speakers , and all age groups



# v4 onwards use new dataset layout

# To do 

# create dataset 

# download robust audio samples

# create tensorflow model

# create json dataset file , and dataloader

# take maximum_frames as 2500 

# ________________________________________________________

# ideas 

# use MFCC, total energy and F0






In [1]:
import sys

sys.path.insert(0, '/media/saurabh/New Volume/tf_audio_sentiment/pyAudioAnalysis')

from  pyAudioAnalysis  import audioBasicIO
from  pyAudioAnalysis import audioFeatureExtraction

In [ ]:
[Fs, x] = audioBasicIO.readAudioFile("/media/saurabh/New Volume/tf_audio_sentiment/data/lizzie.wav");
F = audioFeatureExtraction.stFeatureExtraction(x, Fs, 0.050*Fs, 0.025*Fs);

print(len(F[0,:]))

In [2]:
import tensorflow as tf
from python_speech_features import mfcc
from python_speech_features import delta
from python_speech_features import logfbank
from tensorflow.contrib import rnn
import scipy.io.wavfile as wav
import numpy as np
import os

unhappy_directory_path='/media/saurabh/New Volume/tf_audio_sentiment/data_test/data/0/'
unhappy_directory = os.fsencode(unhappy_directory_path)

happy_directory_path='/media/saurabh/New Volume/tf_audio_sentiment/data/'
happy_directory = os.fsencode(happy_directory_path)

model_path = "/media/saurabh/New Volume/tf_audio_sentiment/models/rnn_model.ckpt"

In [3]:
def audio_to_mfcc(fileurl):
    rate, sig = wav.read(fileurl)
    mfcc_feat = mfcc(sig,rate)
#d_mfcc_feat = delta(mfcc_feat, 2)
#fbank_feat = logfbank(sig,rate)

    return mfcc_feat

In [4]:
# Parameters
learning_rate = 0.001
training_epochs = 25
batch_size = 1
display_step = 1

In [5]:
timesteps=2000 # max timesteps
num_input = 13 # size of vector, at each timestep
num_classes = 2 # happy or sad

num_hidden = 110 # hidden layer num of features

x = tf.placeholder(tf.float32, [None,timesteps, num_input]) 
y = tf.placeholder(tf.int32, [None]) #  Index of output , 0 = sad , 1 = happy

In [6]:
# define a function, that pads audio so that audio frames = max frames

def pad(input):
   # print(input.shape[0])
     if input.shape[0] < timesteps:
        
        diff = timesteps - input.shape[0]
        
        # pad and return input
        return np.pad(input,((0,diff),(0,0)), mode="constant")
    
     elif input.shape[0] > timesteps:
        
        return input[:timesteps,:]
        

In [7]:
# Define weights
weights = {
    # Hidden layer weights => 2*n_hidden because of forward + backward cells
    'out': tf.Variable(tf.random_normal([2*num_hidden, num_classes]))
}
biases = {
    'out': tf.Variable(tf.random_normal([num_classes]))
}


In [8]:
def BiRNN(x, weights, biases):

    # Prepare data shape to match `rnn` function requirements
    # Current data input shape: (batch_size, timesteps, n_input)
    # Required shape: 'timesteps' tensors list of shape (batch_size, num_input)

    # Unstack to get a list of 'timesteps' tensors of shape (batch_size, num_input)
    x = tf.unstack(x, timesteps, 1)

    # Define lstm cells with tensorflow
    # Forward direction cell
    lstm_fw_cell = rnn.BasicLSTMCell(num_hidden, forget_bias=1.0)
    # Backward direction cell
    lstm_bw_cell = rnn.BasicLSTMCell(num_hidden, forget_bias=1.0)

    # Get lstm cell output
    try:
        outputs, _, _ = rnn.static_bidirectional_rnn(lstm_fw_cell, lstm_bw_cell, x,
                                              dtype=tf.float32)
    except Exception: # Old TensorFlow version only returns outputs not states
        outputs = rnn.static_bidirectional_rnn(lstm_fw_cell, lstm_bw_cell, x,
                                        dtype=tf.float32)
        
    # Linear activation, using rnn inner loop last output
    return tf.matmul(outputs[-1], weights['out']) + biases['out']

In [9]:
Logits = BiRNN(x, weights, biases)
# prediction = tf.nn.softmax(logits) # use only for inference, softmax included in loss function


In [10]:
# Minimize error using cross entropy
#cost = tf.reduce_mean(-tf.reduce_sum(y_one_hot*tf.log(pred), reduction_indices=1))

cost = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y, logits=Logits)

# Gradient Descent
optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(cost)

# Initialize the variables (i.e. assign their default value)
init = tf.global_variables_initializer()

In [11]:
# 'Saver' op to save and restore all the variables
saver = tf.train.Saver()

In [15]:
# Start training
with tf.Session() as sess:
    sess.run(init)

    # Training cycle
    for epoch in range(training_epochs):
        avg_cost = 0.
        
        
         # Loop over all files
        for file in os.listdir(unhappy_directory):
            filename = os.fsdecode(file)
            if filename.endswith(".wav"): 
                 print(filename)
                 accoustic_features=pad(audio_to_mfcc(unhappy_directory_path + filename))
                 
                 
                    
                 print(accoustic_features.shape)
                
                 
            
                # train single "unhappy" example
                 _, c = sess.run([optimizer, cost], feed_dict={x: [accoustic_features],
                                                          y: [0]})
            
                 
                 print(c)    
        
                 continue
            else:
                continue
        
        
     
            
           
        # Display logs per epoch step
      #  if (epoch+1) % display_step == 0:
       #     print ("Epoch:", '%04d' % (epoch+1), "cost=", "{:.9f}".format(avg_cost))

    
    print ("Optimization Finished!")
    
    # Save model weights to disk
    save_path = saver.save(sess, model_path)
    print("Model saved in file: %s" % save_path)
          

    # Test model
 #   correct_prediction = tf.equal(tf.argmax(pred, 1), tf.argmax(y, 1))
    # Calculate accuracy for 3000 examples
  #  accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
  #  print "Accuracy:", accuracy.eval({x: mnist.test.images[:3000], y: mnist.test.labels[:3000]})

Raanjhanaa Last Scene Dialogue by Dhanush Heart Touching and Sad007.wav
(2000, 13)
[ 0.16887298]
Raanjhanaa Last Scene Dialogue by Dhanush Heart Touching and Sad016.wav
(2000, 13)
[ 0.1645638]
Raanjhanaa Last Scene Dialogue by Dhanush Heart Touching and Sad025.wav
(2000, 13)
[ 0.16045432]
Raanjhanaa Last Scene Dialogue by Dhanush Heart Touching and Sad000.wav
(2000, 13)
[ 0.15653135]
Raanjhanaa Last Scene Dialogue by Dhanush Heart Touching and Sad001.wav
(2000, 13)
[ 0.15278317]
Raanjhanaa Last Scene Dialogue by Dhanush Heart Touching and Sad002.wav
(2000, 13)
[ 0.14919874]
Raanjhanaa Last Scene Dialogue by Dhanush Heart Touching and Sad003.wav
(2000, 13)
[ 0.14576796]
Raanjhanaa Last Scene Dialogue by Dhanush Heart Touching and Sad004.wav
(2000, 13)
[ 0.14248148]
Raanjhanaa Last Scene Dialogue by Dhanush Heart Touching and Sad005.wav
(2000, 13)
[ 0.1393308]
Raanjhanaa Last Scene Dialogue by Dhanush Heart Touching and Sad006.wav
(2000, 13)
[ 0.13630801]
Raanjhanaa Last Scene Dialogue b

[ 0.04983623]
Raanjhanaa Last Scene Dialogue by Dhanush Heart Touching and Sad011.wav
(2000, 13)
[ 0.04940085]
Raanjhanaa Last Scene Dialogue by Dhanush Heart Touching and Sad012.wav
(2000, 13)
[ 0.04897266]
Raanjhanaa Last Scene Dialogue by Dhanush Heart Touching and Sad013.wav
(2000, 13)
[ 0.04855156]
Raanjhanaa Last Scene Dialogue by Dhanush Heart Touching and Sad014.wav
(2000, 13)
[ 0.04813732]
Raanjhanaa Last Scene Dialogue by Dhanush Heart Touching and Sad015.wav
(2000, 13)
[ 0.04772973]
Raanjhanaa Last Scene Dialogue by Dhanush Heart Touching and Sad017.wav
(2000, 13)
[ 0.04732868]
Raanjhanaa Last Scene Dialogue by Dhanush Heart Touching and Sad018.wav
(2000, 13)
[ 0.04693407]
Raanjhanaa Last Scene Dialogue by Dhanush Heart Touching and Sad019.wav
(2000, 13)
[ 0.04654567]
Raanjhanaa Last Scene Dialogue by Dhanush Heart Touching and Sad020.wav
(2000, 13)
[ 0.04616339]
Raanjhanaa Last Scene Dialogue by Dhanush Heart Touching and Sad021.wav
(2000, 13)
[ 0.0457871]
Raanjhanaa Last S

[ 0.02815795]
Raanjhanaa Last Scene Dialogue by Dhanush Heart Touching and Sad026.wav
(2000, 13)
[ 0.0280111]
Raanjhanaa Last Scene Dialogue by Dhanush Heart Touching and Sad027.wav
(2000, 13)
[ 0.02786584]
Raanjhanaa Last Scene Dialogue by Dhanush Heart Touching and Sad028.wav
(2000, 13)
[ 0.02772196]
Raanjhanaa Last Scene Dialogue by Dhanush Heart Touching and Sad029.wav
(2000, 13)
[ 0.02757956]
Raanjhanaa Last Scene Dialogue by Dhanush Heart Touching and Sad030.wav
(2000, 13)
[ 0.02743842]
Raanjhanaa Last Scene Dialogue by Dhanush Heart Touching and Sad031.wav
(2000, 13)
[ 0.02729877]
Raanjhanaa Last Scene Dialogue by Dhanush Heart Touching and Sad032.wav
(2000, 13)
[ 0.02716049]
Raanjhanaa Last Scene Dialogue by Dhanush Heart Touching and Sad033.wav
(2000, 13)
[ 0.02702346]
Raanjhanaa Last Scene Dialogue by Dhanush Heart Touching and Sad034.wav
(2000, 13)
[ 0.0268877]
Raanjhanaa Last Scene Dialogue by Dhanush Heart Touching and Sad035.wav
(2000, 13)
[ 0.02675331]
Raanjhanaa Last Sc

[ 0.01937395]
Raanjhanaa Last Scene Dialogue by Dhanush Heart Touching and Sad000.wav
(2000, 13)
[ 0.01930192]
Raanjhanaa Last Scene Dialogue by Dhanush Heart Touching and Sad001.wav
(2000, 13)
[ 0.01923036]
Raanjhanaa Last Scene Dialogue by Dhanush Heart Touching and Sad002.wav
(2000, 13)
[ 0.01915937]
Raanjhanaa Last Scene Dialogue by Dhanush Heart Touching and Sad003.wav
(2000, 13)
[ 0.01908885]
Raanjhanaa Last Scene Dialogue by Dhanush Heart Touching and Sad004.wav
(2000, 13)
[ 0.01901879]
Raanjhanaa Last Scene Dialogue by Dhanush Heart Touching and Sad005.wav
(2000, 13)
[ 0.01894931]
Raanjhanaa Last Scene Dialogue by Dhanush Heart Touching and Sad006.wav
(2000, 13)
[ 0.0188803]
Raanjhanaa Last Scene Dialogue by Dhanush Heart Touching and Sad008.wav
(2000, 13)
[ 0.01881163]
Raanjhanaa Last Scene Dialogue by Dhanush Heart Touching and Sad009.wav
(2000, 13)
[ 0.01874354]
Raanjhanaa Last Scene Dialogue by Dhanush Heart Touching and Sad010.wav
(2000, 13)
[ 0.01867591]
Raanjhanaa Last S

KeyboardInterrupt: 

In [16]:
accoustic_features=audio_to_mfcc( "/home/saurabh/Documents/audio_classification/data/boy.wav")

print(accoustic_features.shape)

new_features = pad(accoustic_features)

print(new_features.shape)

(999, 13)
(2000, 13)
